# Creating Dataset





> 긍정, 부정 감성 분석



감성 분석은 입력된 자연어 안의 주관적 의견, 감정 등을 찾아내는 문제이다. 이 가운데 극성(polarity) 감서 분석은 문장의 긍정/부정이나 긍정/중립/부정을 분류한다.
<br>

예제를 통해 <Naver sentiment movie corpus v1.0> 코퍼스를 통해 긍정/부정 감성 분석을 해보자.

In [257]:
import pandas as pd
import numpy as np

In [258]:
!git clone https://github.com/e9t/nsmc.git 

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [290]:
train_text = open("nsmc/ratings_train.txt", 'rb').read().decode(encoding='utf-8')
test_text = open("nsmc/ratings_test.txt", 'rb').read().decode(encoding='utf-8')

In [262]:
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


In [162]:
train[0:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [164]:
test[0:5]

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0




*   처음의 id는 각 데이터의 고유한 아이디이고, document는 실제 리뷰 내용
*   label은 긍정/부정 나타내는 값으로, 0은 부정, 1은 긍정



# Preprocessing

In [261]:
# 자연어 토큰화 및 정제

# 정규표현식 라이브러리 불러오기
import re 

def clean_str(string):
    string = re.sub(r"[^가-힣A-Za-z0-9(),!?\'\`]", " ", string)     # 한글, 영문, 숫자, 괄호, 쉼표, 느낌표, 물음표, 작음따옴표, 역따옴표 제외한 나머지 모두 찾아서 공백(" ")으로 바꾸기
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " ( ", string) 
    string = re.sub(r"\)", " ) ", string) 
    string = re.sub(r"\?", " ? ", string) 
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"\s'{2,}", " '", string)
    string = re.sub(r"\'", "", string)                          

    return string.lower()


def preprocessing(data):
  text = [clean_str(str(sentence)) for sentence in data['document']]
  data['document'] = pd.DataFrame(text, columns=['document'])
  return data

In [264]:
preprocessing(train)[0:5]

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 ! 스파이더맨에서 늙어보이기만 했던 커스...,1


In [296]:
# torchtext에서는 csv 형태의 데이터를 받기 때무에 전처리한 데이터 txt로 저장
preprocessing(train).to_csv("/content/train_set.txt",sep='\t', index=False)
preprocessing(test).to_csv("/content/test_set.txt",sep='\t',index=False)

# Tokenization

In [265]:
!git clone https://gist.github.com/40eea22ef4a89f629abd87eed535ac6a.git

Cloning into '40eea22ef4a89f629abd87eed535ac6a'...
remote: Enumerating objects: 3, done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (3/3), done.


In [266]:
stopwords = open('40eea22ef4a89f629abd87eed535ac6a/stopwords-ko.txt', mode='rt', encoding='utf-8')
stopwords = stopwords.read().split()

In [141]:
!pip install konlpy

In [267]:
# Konlpy 토크나이저 불러오기
from konlpy.tag import Okt
okt = Okt()

In [293]:
# 필드 정의
ID = data.Field(sequential = False,
                use_vocab = False) # 실제 사용은 하지 않을 예정

TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize = okt.morphs, # 토크나이저로는 okt 사용.
                  stop_words = stopwords,
                  lower=True,
                  batch_first=True,
                  fix_length=30)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   is_target=True)

In [297]:
from torchtext.data import TabularDataset
train_data, test_data = TabularDataset.splits(
        path='.', train='/content/train_set.txt', test='/content/test_set.txt', format='tsv',
        fields=[('id', ID), ('text', TEXT), ('label', LABEL)], skip_header=True)

In [298]:
print(vars(train_data[0]))

{'id': '9976970', 'text': ['더빙', '진짜', '짜증나네요', '목소리'], 'label': '0'}


In [299]:
# vocab 만들기
TEXT.build_vocab(train_data, min_freq=10, max_size=25000)

In [302]:
print(TEXT.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fa3744f2ea0>, {'<unk>': 0, '<pad>': 1, '영화': 2, '!': 3, ',': 4, '도': 5, '?': 6, '는': 7, '은': 8, '너무': 9, '다': 10, '정말': 11, '적': 12, '진짜': 13, '점': 14, '연기': 15, '평점': 16, '최고': 17, '내': 18, '잘': 19, '스토리': 20, '생각': 21, '못': 22, '드라마': 23, '1': 24, '게': 25, '이다': 26, '감동': 27, '사람': 28, '보고': 29, '말': 30, '고': 31, '더': 32, '배우': 33, '거': 34, '감독': 35, '그냥': 36, '요': 37, '본': 38, '재미': 39, '내용': 40, '뭐': 41, '중': 42, '쓰레기': 43, '보다': 44, '없는': 45, '지': 46, '10': 47, '봤는데': 48, '작품': 49, '사랑': 50, '볼': 51, '마지막': 52, '2': 53, '이건': 54, '좋은': 55, '정도': 56, '입니다': 57, '완전': 58, '있는': 59, '처음': 60, '장면': 61, '액션': 62, '3': 63, '주인공': 64, '대': 65, '걸': 66, '보는': 67, '개': 68, '최악': 69, '지금': 70, '돈': 71, '이야기': 72, '별로': 73, '임': 74, '봐도': 75, '느낌': 76, '연출': 77, '끝': 78, '명작': 79, '듯': 80, '인데': 81, '별': 82, '역시': 83, '기': 84, '난': 85, '많이': 86, '재밌게': 87, '서': 88, '이해': 89, '이영화': 90, '면': 91, '라고': 92, '이나': 93, '여자': 94, '때문': 95, '성': 96, '

# pre-trained Word2vec으로 학습시키기

In [ ]:
'''
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim
'''

In [303]:
import gensim
word2vec_model = gensim.models.Word2Vec.load('/content/ko.bin')

In [309]:
n=0
for i in model.wv.vocab:
  if n <50 : 
    print(i)
    n +=1

관위
정어리
유식론
장로회
춘추관
도입부
민병
어렵
매니저
청담
광주시
고민
어로
어록
폴라리스
십만
관음
효과적
관의
당뇨병
영중추
듣
민법
마두라
중국계
여덟째
ㄴ단
ㄴ다
테스트
어찌하
정안국
엎드리
싱가포르
벡터
관자
관작
팽대부
테이블
분석학
유장
렙톤
어뢰
다다르
제공자
히트
멸망
암기
마르텔
뻔
윈도


In [301]:
print(model.wv.vectors) 

[[ 1.2513299e+00 -7.9136038e-01 -1.7325785e+00 ...  2.3886933e+00
  -1.4849446e+00  2.9343410e+00]
 [-9.2375271e-02 -5.4772741e-01  3.0446327e-01 ...  1.0542500e+00
   1.0875396e+00 -9.1070002e-01]
 [ 1.3803720e-01 -5.0465461e-02 -2.1609845e+00 ...  2.4863910e-03
  -8.3771986e-01 -1.2966256e+00]
 ...
 [-8.4928520e-02 -1.0080223e-01  9.7347349e-02 ... -1.7621946e-01
   1.3997090e-01 -1.7076550e-01]
 [ 5.3990629e-02 -2.1882482e-01 -8.0515213e-02 ...  2.7542275e-01
  -1.2152838e-01 -2.4455969e-01]
 [ 5.4298151e-01  2.0541237e-01  1.1461087e-01 ... -8.8693380e-02
   3.0509625e-02 -1.9296505e-01]]


In [226]:
vocab_size = len(TEXT.vocab.stoi) + 1
vocab_size

10003

In [232]:
embedding_matrix = np.zeros((vocab_size, 200))
# 단어 집합 크기의 행과 300개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
np.shape(embedding_matrix)

(10003, 200)

In [233]:
def get_vector(word):
    if word in model:
        return model[word]
    else:
        return None

In [234]:
for word, i in TEXT.vocab.stoi.items(): # 훈련 데이터의 단어 집합에서 단어와 정수 인덱스를 1개씩 꺼내온다.
    temp = get_vector(word) # 단어(key) 해당되는 임베딩 벡터의 300개의 값(value)를 임시 변수에 저장
    if temp is not None: # 만약 None이 아니라면 임베딩 벡터의 값을 리턴받은 것이므로
        embedding_matrix[i] = temp # 해당 단어 위치의 행에 벡터의 값을 저장한다.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [236]:
print(model['추천'])

[-0.21245585 -0.76455975  1.2335763  -2.0687602   0.11830104 -0.54881305
 -1.1947731   0.3467694  -1.0311009  -0.3603907   1.3440485  -0.4072002
  1.1131706  -0.9551345  -0.7064215  -0.1686603  -0.58696073 -1.4859254
  0.05223045 -0.23236327  0.6323806  -1.1020958  -1.2635112   0.6220531
  0.17838949 -0.70615005 -1.3628044  -1.4818763  -0.18965532  0.11690602
 -0.09560227 -0.39899632  1.0731212  -0.63874424  0.017969    0.98583263
  1.3941766   0.08089745 -0.81685823 -0.7022943  -0.5008832  -0.54513174
  0.34584457 -1.7315279  -0.24291196  0.6328651   1.4537793  -0.7011548
  0.80850524  0.74069935 -0.9272299  -1.335555   -1.1335523   3.2529519
  0.6823866  -1.4942434   3.213415    1.0216335  -0.4152105  -0.11520938
 -0.08406533 -0.49404764 -1.7769778   0.19683318  0.48151487 -0.67674905
 -1.5347718   0.17763563 -0.6901806  -0.71837485  0.26211363  1.3736591
 -0.81411374  1.5484174  -0.32827973 -1.2308834  -0.24815056 -0.05604961
  2.4168842  -1.3484128  -1.9251053   0.85578465 -1.82561

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [256]:
print('단어 nice의 정수 인덱스 :', TEXT.vocab.stoi['추천'])

단어 nice의 정수 인덱스 : 162


In [238]:
print(embedding_matrix[162])

[-0.21245585 -0.76455975  1.2335763  -2.06876016  0.11830104 -0.54881305
 -1.19477308  0.34676939 -1.03110087 -0.36039069  1.3440485  -0.40720019
  1.11317062 -0.95513451 -0.70642149 -0.1686603  -0.58696073 -1.48592544
  0.05223045 -0.23236327  0.6323806  -1.10209584 -1.26351118  0.62205309
  0.17838949 -0.70615005 -1.36280441 -1.48187625 -0.18965532  0.11690602
 -0.09560227 -0.39899632  1.07312119 -0.63874424  0.017969    0.98583263
  1.3941766   0.08089745 -0.81685823 -0.70229429 -0.50088322 -0.54513174
  0.34584457 -1.73152792 -0.24291196  0.63286507  1.45377934 -0.70115483
  0.80850524  0.74069935 -0.92722988 -1.33555496 -1.13355231  3.25295186
  0.68238658 -1.49424338  3.21341491  1.02163351 -0.41521049 -0.11520938
 -0.08406533 -0.49404764 -1.77697778  0.19683318  0.48151487 -0.67674905
 -1.5347718   0.17763563 -0.6901806  -0.71837485  0.26211363  1.37365913
 -0.81411374  1.54841745 -0.32827973 -1.23088336 -0.24815056 -0.05604961
  2.41688418 -1.34841275 -1.92510533  0.85578465 -1

# Dataloader



In [243]:
batch_size = 5

In [241]:
from torchtext.data import Iterator

In [244]:
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
test_loader = Iterator(dataset=test_data, batch_size = batch_size)

In [312]:
batch = next(iter(train_loader)) # 첫번째 미니배치
print(batch.text)
print(batch.label)

tensor([[2677,   13, 3507, 1289,   64,    2,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [ 544,    4,  480,  922,  505,  389,    0,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [  39,   18,    6,  285,   24,    6,  285,  226,  156,    6,  285, 5499,
          438,   31,    0,  438,  286,  213,   57,   25],
        [  13, 3563,  143,    2,    2,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [   0,    9,    0,   11, 4113,  301,   19,    9, 1400,   25,  540, 1182,
            0,    9, 1400,   35, 8351,  917,  519,    0]])
tensor([1, 1, 1, 0, 1])
